<a href="https://colab.research.google.com/github/hbthekraken/Job_Shift_AssignmentProject/blob/main/Job_Shift_Assigner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this code, there is a function which can build concrete Pyomo models. These models can assign jobs that are sequentially processed over a series of processors to the shifts, so they model the planning. These models can also support parallelism for machines. Each job represents a fixed quantity (lot) of parts of the same type that can be done in a shift.

Due dates constraints, sequentiality constraints, parallelism constraints etc. are therefore available. With these, the model can be tested (and used) by generating a hypothetical list of parts to be made and then converting them into jobs. Conversion is generally needed because each job must represent what can be done in one shift but more parts might be needed than that.

In [1]:
import pyomo.environ as pyo
import highspy as hp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


plt.style.use("ggplot")

Calling necessary libraries for modelling and testing.

In [2]:
slvr = "appsi_highs"
print(pyo.SolverFactory(slvr).available())

True


Here, we check if the HiGHS solver is available. If it is not, another available solver must be supplied through the variable ```slvr```.




In [3]:
def build_concr_model(N, K, L, J_NL, alpha_coefficients, d_dates, p_times, sigma_availabilities):
  model_ = pyo.ConcreteModel()

  # Some model parameters related to index sets
  model_.N = pyo.Param(within=pyo.PositiveIntegers, initialize=N)
  model_.K = pyo.Param(within=pyo.PositiveIntegers, initialize=K)
  model_.L = pyo.Param(within=pyo.PositiveIntegers, initialize=L)

  # model index sets here, J_NL being the  subset of J
  model_.J = pyo.RangeSet(1, model_.N)
  model_.S = pyo.RangeSet(1, model_.K)
  model_.M = pyo.RangeSet(1, model_.L)

  model_.J_NL = pyo.Set(within=model_.J, initialize=J_NL)

  # Decision Variables which are all binary
  model_.x = pyo.Var(model_.S, model_.J, model_.M, within=pyo.Binary)

  # Actual model parameters related to the problem
  model_.alpha = pyo.Param(model_.J, within=pyo.PositiveReals, initialize=alpha_coefficients) # Importance Factors - coefficients
  model_.d = pyo.Param(model_.J, within=pyo.PositiveReals, initialize=d_dates) # Due dates for Not-Late jobs, although indexed for all j in J, only the ones for the j in J_NL will be used
  model_.p = pyo.Param(model_.J, model_.M, within=pyo.Binary, initialize=p_times) # Processing time for job j on processor/machine on m
  model_.sigma = pyo.Param(model_.S, model_.M, within=pyo.NonNegativeReals, initialize=sigma_availabilities) # Availability (can be > 1 if parallel machines exist) of processor-m on shift-s

  # Model Objective
  def z(_model):
    return sum(_model.alpha[_j] * sum(_s * _model.x[_s, _j, _model.L] for _s in _model.S) for _j in _model.J)

  model_.objective = pyo.Objective(rule=z, sense=pyo.minimize)

  # Constraints:
  # Enforcing due dates:
  def EnforceDueDates(_model, _j_NL):
    return sum(_s * _model.x[_s, _j_NL, _model.L] for _s in _model.S) <= _model.d[_j_NL]

  # Machining time constraint
  def MachineTimeConstraint(_model, _s, _m):
    return sum(_model.p[_j, _m] * _model.x[_s, _j, _m] for _j in _model.J) <= _model.sigma[_s, _m]

  # One Shift for One process Constraint
  def OneShiftConstraint(_model, _j, _m):
    return sum(_model.x[_s, _j, _m] for _s in _model.S) == 1

  # Sequantieal Machining Constraint
  def SequentialityConstraint(_model, _j, _m):
    if _m < _model.L:
      return _model.p[_j, _m] + sum(_s * (_model.x[_s, _j, _m] - _model.x[_s, _j, _m + 1]) for _s in _model.S) <= 0.0
    else:
      return pyo.Constraint.Skip

  model_.EnforceDueDates = pyo.Constraint(model_.J_NL, rule=EnforceDueDates)
  model_.MachineTimeLimit = pyo.Constraint(model_.S, model_.M, rule=MachineTimeConstraint)
  model_.OneShiftRule = pyo.Constraint(model_.J, model_.M, rule=OneShiftConstraint)
  model_.SequentialityConstraint = pyo.Constraint(model_.J, model_.M, rule=SequentialityConstraint)
  return model_

This is the function that takes the data (parameters of the model) as input and outputs a concrete Pyomo model. The details of the model and the program can be found at [the Github repo of the project](https://github.com/hbthekraken/Job_Shift_AssignmentProject).

In [13]:
def prod_to_jobs(prod_df):
  jobs = {}
  N = len(prod_df.index)
  job_count = 0
  product_count = 0
  while N > 0:
    name = prod_df.index[product_count]
    a_product = prod_df.iloc[product_count]
    Qty, LS, DT, alpha = tuple(a_product)
    Qty, LS = int(Qty), int(LS)
    if Qty % LS == 0:
      k = Qty // LS
      for i in range(k):
        jobs[job_count] = (str(name) + f'_{i}', int(LS), int(DT), alpha, job_count) # for a job, in the strict technical sense,
        # processing time, due time, importance factor, and its assigned shift are required to be defined
        # name and part count are present for convenience
        job_count += 1
    elif Qty < LS:
      if Qty / LS >= 0.5 and alpha > 1.5:
        jobs[job_count] = (str(name), int(LS), int(DT), alpha, job_count)
      else:
        pass
    elif Qty > LS:
      if alpha >= 1.0:
        k = 1 + (Qty // LS)
        for i in range(k):
          jobs[job_count] = (str(name) + f'_{i}', int(LS), int(DT), alpha, job_count)
          job_count += 1
      else:
        k = Qty // LS
        for i in range(k):
          jobs[job_count] = (str(name) + f'_{i}', int(LS), int(DT), alpha, job_count)
          job_count += 1
    else:
      raise Exception("Unknown error.")

    N -= 1
    product_count += 1
  return pd.DataFrame(jobs, index=["OpName", "LotSize", "DueTime", "Imp.Factor", "AssignedShifts"]).transpose()

This function ```prod_to_jobs``` takes a Pandas Dataframe that contains a list of products that have the following attributes:
1. Quantity: The quantity of the product requested.
2. Lot size: The number of products that can be produced in one shift with one processor.
3. Due time: Products production completion due date in shifts. E.g., if there are 3 shifts per day and 5 days before the completion is due, then due time of the product is 15 shifts.
4. Importance Factor: Apart from the due times, an importance factor helps planning process to prioritize some products' production before others. This can also be used to make products that are late already to be produced first by giving them a very large coefficients.

Then it outputs another DataFrame that contains a list of jobs and their parameters that can be used for the job-specific parameters.
Shift specific parameters are generally required to be given during the actual production planning.


In [14]:
gen = np.random.default_rng()
test_product_number = 10
# all test data are generated here randomly
test_data = np.array([
    [gen.integers(low=1, high=19), gen.integers(low=1, high=16), gen.integers(low=1, high=26), gen.uniform(0.1, 5.0)] for _ in range(test_product_number)])

# These numbers are used to 'name' the products, actual names of the products can take this place
prod_numbers = np.arange(100, 1000)
gen.shuffle(prod_numbers)
prod = pd.DataFrame(test_data, columns=["Quantity", "LotSize", "DueTime", "ImportanceFactor"], index=[f'Part--{prod_numbers[i]}' for i in range(len(test_data))])
print(prod)

           Quantity  LotSize  DueTime  ImportanceFactor
Part--649      12.0      7.0      5.0          1.734115
Part--100      17.0     11.0      6.0          4.092460
Part--566      11.0      1.0     22.0          1.518779
Part--267       2.0      4.0     11.0          4.365785
Part--556       9.0     14.0     15.0          3.002049
Part--694       7.0      6.0      5.0          3.406632
Part--987       6.0      6.0     10.0          3.818443
Part--643       9.0      9.0     12.0          2.262658
Part--577      14.0     14.0     15.0          3.822975
Part--575       9.0     12.0     11.0          0.190099


Above block generates and prints some test data, a DataFrame of products that have the aforementioned attributes.

In [32]:
jobs_df = prod_to_jobs(prod)
jobs_df.iloc[-10:, :]

,OpName,LotSize,DueTime,Imp.Factor,AssignedShifts
10,Part--566_6,1,22,1.518779,10
11,Part--566_7,1,22,1.518779,11
12,Part--566_8,1,22,1.518779,12
13,Part--566_9,1,22,1.518779,13
14,Part--566_10,1,22,1.518779,14
15,Part--694_0,6,5,3.406632,15
16,Part--694_1,6,5,3.406632,16
17,Part--987_0,6,10,3.818443,17
18,Part--643_0,9,12,2.262658,18
19,Part--577_0,14,15,3.822975,19


This block shows the list of jobs that are created from the above products data. It is formed by the function ```prod_to_jobs```.

In [33]:
number_of_jobs = len(jobs_df.OpName)
number_of_shifts = np.max(jobs_df.AssignedShifts)
number_of_machines = 4

importance_factors = jobs_df["Imp.Factor"].values
due_dates = jobs_df["DueTime"].values
proc_time = np.ones((number_of_jobs, number_of_machines,), dtype=int)
available_machines = 2 * np.ones((number_of_shifts, number_of_machines,), dtype=int)
due_enforced = tuple() # Empty set for now, so no due times are enforced

Jobs2Plan_model = build_concr_model(number_of_jobs, number_of_shifts, number_of_machines,
                                    due_enforced,
                                    {i + 1 : importance_factors[i] for i in range(importance_factors.shape[0])},
                                    {i + 1 : due_dates[i] for i in range(due_dates.shape[0])},
                                    {(i + 1, j + 1) : proc_time[i, j] for i in range(proc_time.shape[0]) for j in range(proc_time.shape[1])},
                                    {(i + 1, j + 1) : available_machines[i, j] for i in range(available_machines.shape[0]) for j in range(available_machines.shape[1])}
                                    )

Here, the parameters are formed and fed into the concrete model builder function. It builds our BILP model that models the optimal assignment problem.

In [34]:
opt = pyo.SolverFactory(slvr)
resulting_plan = opt.solve(Jobs2Plan_model, tee=True)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 216 rows; 1520 cols; 5320 nonzeros; 1520 integer variables (1520 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [2e+00, 8e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+00]
Presolving model
216 rows, 1520 cols, 5320 nonzeros  0s
204 rows, 1280 cols, 4360 nonzeros  0s

Solving MIP model with:
   204 rows
   1280 cols (1280 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   4360 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work     

Above, the solver is called to solve the model. All details of the solution are printed since the argument ```tee``` is ```True```.

In [38]:
optimal_sol = Jobs2Plan_model.x.extract_values()

for _j in range(number_of_jobs):
  assigned_shift = "("
  for _m in range(number_of_machines):
    s = sum([(_s + 1) * round(optimal_sol[(_s +  1, _j + 1, _m + 1)]) for _s in range(number_of_shifts)])
    assigned_shift += f"{s}, "
  assigned_shift += ")"
  jobs_df.loc[_j, "AssignedShifts"] = assigned_shift

print(jobs_df)

          OpName LotSize DueTime Imp.Factor      AssignedShifts
0    Part--649_0       7       5   1.734115      (5, 6, 7, 8, )
1    Part--649_1       7       5   1.734115      (4, 5, 6, 7, )
2    Part--100_0      11       6    4.09246      (1, 2, 3, 4, )
3    Part--100_1      11       6    4.09246      (1, 2, 3, 4, )
4    Part--566_0       1      22   1.518779     (7, 8, 9, 10, )
5    Part--566_1       1      22   1.518779  (10, 11, 12, 13, )
6    Part--566_2       1      22   1.518779   (9, 10, 11, 12, )
7    Part--566_3       1      22   1.518779   (9, 10, 11, 12, )
8    Part--566_4       1      22   1.518779    (8, 9, 10, 11, )
9    Part--566_5       1      22   1.518779      (6, 7, 8, 9, )
10   Part--566_6       1      22   1.518779  (10, 11, 12, 13, )
11   Part--566_7       1      22   1.518779      (6, 7, 8, 9, )
12   Part--566_8       1      22   1.518779      (5, 6, 7, 8, )
13   Part--566_9       1      22   1.518779     (7, 8, 9, 10, )
14  Part--566_10       1      22   1.518

Here, the optimal solution is converted back to meaningful data and it is merged into the DataFrame of jobs. So, the shifts each job assigned to in the optimal plan can be seen in the last column.